In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn import model_selection, metrics
import sci

In [2]:
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

In [3]:
df = pd.read_csv('/Users/JQC/Desktop/ml-100k/u.data', sep='\t', \
                 names=['user_id','item_id','rating','timestamp'])
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [14]:
item = df.groupby(by=['item_id', 'user_id']).count()
item.head()


rating  timestamp
item_id user_id                   
1       1             1          1
        2             1          1
        5             1          1
        6             1          1
        10            1          1

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
user_id      100000 non-null int64
item_id      100000 non-null int64
rating       100000 non-null int64
timestamp    100000 non-null int64
dtypes: int64(4)
memory usage: 3.1 MB


In [60]:
len(df.user_id.unique())

943

In [56]:
n_users = df.user_id.nunique()
n_items = df.item_id.nunique()

In [58]:
n_users, n_items

(943, 1682)

In [44]:
item_for_user = {item_id: item2.loc[df.item_id == item_id, 'user_id'].tolist()\
                 for item_id in item2.item_id.unique()}
item_for_user

{1: [57,
  13,
  805,
  63,
  776,
  802,
  887,
  927,
  542,
  294,
  1,
  276,
  609,
  417,
  540,
  13,
  217,
  792,
  868,
  655,
  394,
  918,
  42,
  69,
  7,
  627,
  145,
  887,
  922,
  16,
  889,
  588,
  774,
  313,
  763,
  495,
  280,
  178,
  346,
  457,
  903,
  738,
  916,
  747,
  648,
  705,
  445,
  291,
  865,
  28,
  334,
  396,
  521,
  707,
  193,
  537,
  174,
  303,
  137,
  931,
  445,
  87,
  878,
  59,
  722,
  222,
  504,
  90,
  313,
  562,
  144,
  571,
  279,
  893,
  274,
  596,
  5,
  385,
  883,
  452,
  7,
  711,
  795,
  804,
  398,
  59,
  92,
  200,
  527,
  425,
  883,
  95,
  449,
  457,
  42,
  45,
  213,
  493,
  207,
  458,
  573,
  268,
  315,
  177,
  747,
  916,
  747,
  41,
  212,
  498,
  426,
  528,
  943,
  615,
  934,
  250,
  343,
  442,
  409,
  788,
  272,
  340,
  777,
  425,
  660,
  686,
  298,
  301,
  666,
  693,
  897,
  7,
  338,
  777,
  535,
  389,
  878,
  406,
  159,
  450,
  921,
  94,
  442,
  303,
  901,
  757,
  2

In [63]:
user_item_matrix = np.zeros((n_users, n_items))
# 把用户所有的评分的电影对应到矩阵中
for line in df.itertuples():
    user_item_matrix[line[1]-1,line[2]-1] = line[3]

In [65]:
user_item_matrix[:10, :10]

array([[5., 3., 4., 3., 3., 5., 4., 1., 5., 3.],
       [4., 0., 0., 0., 0., 0., 0., 0., 0., 2.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [4., 3., 0., 0., 0., 0., 0., 0., 0., 0.],
       [4., 0., 0., 0., 0., 0., 2., 4., 4., 0.],
       [0., 0., 0., 5., 0., 0., 5., 5., 5., 4.],
       [0., 0., 0., 0., 0., 0., 3., 0., 0., 0.],
       [0., 0., 0., 0., 0., 5., 4., 0., 0., 0.],
       [4., 0., 0., 4., 0., 0., 4., 0., 4., 0.]])

In [66]:
user_similarit_matrix = metrics.pairwise.cosine_similarity(user_item_matrix)

In [73]:
user_similarit_matrix[:5, :5]

array([[1.        , 0.16693098, 0.04745954, 0.06435782, 0.37847518],
       [0.16693098, 1.        , 0.11059132, 0.17812119, 0.07297896],
       [0.04745954, 0.11059132, 1.        , 0.34415072, 0.02124453],
       [0.06435782, 0.17812119, 0.34415072, 1.        , 0.03180425],
       [0.37847518, 0.07297896, 0.02124453, 0.03180425, 1.        ]])

In [101]:
for i in range(n_users):
    user_similarit_matrix[i][i] = 0

In [102]:
user_similarit_matrix[:5, :5]

array([[0.        , 0.16693098, 0.04745954, 0.06435782, 0.37847518],
       [0.16693098, 0.        , 0.11059132, 0.17812119, 0.07297896],
       [0.04745954, 0.11059132, 0.        , 0.34415072, 0.02124453],
       [0.06435782, 0.17812119, 0.34415072, 0.        , 0.03180425],
       [0.37847518, 0.07297896, 0.02124453, 0.03180425, 0.        ]])

In [79]:
# 测试矩阵
user_similarit_matrix2 = np.zeros((n_users, n_items))
for i in range(n_users):
    for j in range(n_users):
        if i == j:
            user_similarit_matrix2[i, j] = 1
        else:
            temp = np.dot(user_item_matrix[i], user_item_matrix[j])
            denom = np.linalg.norm(user_item_matrix[i]) * np.linalg.norm(user_item_matrix[j])
            if denom == 0:
                user_similarit_matrix2[i, j] = 0
            else:
                user_similarit_matrix2[i, j] = temp / denom

In [80]:
user_similarit_matrix2[:5, :5]

array([[1.        , 0.16693098, 0.04745954, 0.06435782, 0.37847518],
       [0.16693098, 1.        , 0.11059132, 0.17812119, 0.07297896],
       [0.04745954, 0.11059132, 1.        , 0.34415072, 0.02124453],
       [0.06435782, 0.17812119, 0.34415072, 1.        , 0.03180425],
       [0.37847518, 0.07297896, 0.02124453, 0.03180425, 1.        ]])

In [103]:
# 基于用户的推荐
# 目标用户
user_index = 196 - 1
user_item_matrix[user_index, :20]

array([0., 0., 0., 0., 0., 0., 0., 5., 0., 0., 0., 0., 2., 0., 0., 0., 0.,
       0., 0., 0.])

In [118]:
# 去出用户相似度最大的10个 输出的这10个相似度比较大的索引
user_similarit_array = np.argsort(user_similarit_matrix[user_index, :])[::-1][:10]
user_similarit_array

array([578, 590, 305, 522, 335, 859,  18, 353, 650, 691])

In [108]:
sorted(user_similarit_matrix[user_index, :], reverse=True)[:10]

[0.34009675216916435,
 0.3264871651928991,
 0.3107314947929877,
 0.2907638540125011,
 0.26686570466071813,
 0.26137165344164665,
 0.2595530193875286,
 0.2518471208979924,
 0.2516877763757853,
 0.2490104069321108]

In [109]:
user_similarit_matrix2[user_index, :][590]

0.3264871651928991

In [154]:
# 这个会输出该用户评分大于4的所有电影所在的列
np.nonzero(user_item_matrix[578, :] > 4)

(array([ 49,  82, 172, 193, 201, 215, 257, 434, 602, 708]),)

In [1]:
recommend_film_ = []
for user in user_similarit_array:
    # (用户所在行, 喜欢电影所在列, 电影评分)
    for col in np.nonzero(user_item_matrix[user, :] > 4)[0]:
        # 如果目标用户看过该电影 跳过
        if user_item_matrix[user_index][col] > 0:
            continue
        recommend_film_.append((user, col, user_item_matrix[user][col]))
# 按照电影评分大小排序
sorted(recommend_film_, key=lambda x:x[2], reverse=True)
recommend_film_[:10]

NameError: name 'user_similarit_array' is not defined

In [162]:
# 上面的改进版 使用列表推导式
recommend_film = []
for user in user_similarit_array:
    # (用户所在行, 喜欢电影所在列, 电影评分)
    user_faver_films = [(user, col, user_item_matrix[user][col]) \
                        for col in np.nonzero(user_item_matrix[user, :] > 4)[0] \
                        if user_item_matrix[user_index][col] == 0]
    recommend_film.extend(user_faver_films)

sorted(recommend_film, key=lambda x:x[2], reverse=True)
# recommend_film
recommend_film == recommend_film_

True

In [156]:
recommend_film == recommend_film_

True

In [178]:
df[(df.user_id == 2) & (df.item_id==1)]

,user_id,item_id,rating,timestamp
26184,2,1,4,888550871


In [111]:
a = set([1, 0, 0, 2, 4, 5])
b = set([2, 0, 0, 6, 8, 10])

In [112]:
a - b

{1, 4, 5}

In [113]:
b - a

{6, 8, 10}